# Plotting Ballistic Ejecta from the South Pole-Aitken Impact

## Workflow:
This script runs through all the tracers in the data file and selects the tracers that follow a ballistic trajectory. It outputs the results to a csv file which can be used for plotting.

In [1]:
#import python packages 
import tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
DATA_PATH = '/data2/tmd02/SPA/v15cold/tracer_export_all.npz'
CSV_NAME = 'full_tracer_v10hot'

In [2]:
#import the data file. Change the path to your file :) 
data = tools.load_ballistic_data(DATA_PATH, end = 397)

In [3]:
# get data
velocity_data = tools.get_velocity(data)
radial_distance_data = tools.get_radial_distance(data)
radial_velocity_data = tools.get_radial_velocity(data)
radial_acceleration_data = tools.get_radial_acceleration(data, radial_velocity_data)
average_time = tools.get_average_time(data)

In [7]:
# tracers = data['trm']
# print(tracers.shape)

# np.sum(tracers == 4.0)

(606129,)


2640

Filter tracers that reach h_launch. Return tracer indexes that pass filter criteria and the corresponding time step that the criteria is first true

In [8]:
h_launch = 150
tracer_idxs, time_steps = tools.BallisticCheckVec(data, h_launch)
# print(time_steps)
# print(tracer_idxs[398])
# print(time_steps.max())
# print(np.min(time_steps))
# print(np.max(time_steps))
# print(time_steps.shape)
ejection_times = data['time'][time_steps]

In [9]:
# get data of filtered tracers
tracer_material = data['trm'][tracer_idxs]
pressure = data['trp'][tracer_idxs]
initial_depth = data['depth'][tracer_idxs]
volume = data['volume'][tracer_idxs]

# trmmm = data['r'][:, 0][:, None]
# print(trmmm.shape)
# r_dist = radial_distance_data['r_cumul'][tracer_idxs, time_steps] 


In [10]:
# calculate relative velocity
R0 = 1750
g0 = 0.00162

g = g0 * R0**2 / (R0 + h_launch) **2
v_tilda = tools.get_v_tilda_vec(velocity_data, tracer_idxs, time_steps, R0 + h_launch, g)

Compute launch parameters. Launch velocity, launch latitude, launch longitude, launch angle, distance traveled, maximum height

In [11]:
v_r, longitude, latitude = tools.get_launch_vector_vec(data,velocity_data, tracer_idxs, time_steps)
longitude_magnitude, longitude_direction = longitude
latitude_magnitude, latitude_direction = latitude

In [12]:
theta_launch, height, distance, azimuth = tools.get_ballistic_vectors_vec(data, velocity_data, tracer_idxs, time_steps, h_launch, R0, g0)

/home/nb3018/SPA/tools/ballistics.py:105: RuntimeWarning: invalid value encountered in arccos
  beta = np.arccos(data['zmark'] / (data['r']+ 1e-12))[:, :-1]  # angle at the centre of the moon


In [18]:
print('depth min =',initial_depth.min())
print('depth max =',initial_depth.max())

depth min = -190.50372518337417
depth max = 127.70442982002646


In [13]:
#multiply the azimuth with -1 to get the right set up
final_latitude, final_longitude = tools.track_vec(latitude[0], longitude[0], -1 * azimuth, distance)

In [14]:
c1 = final_longitude < 0
c2 = final_longitude > 0
final_longitude[c1] = final_longitude[c1] + 180
final_longitude[c2] = final_longitude[c2] - 180
final_latitude = -1 * final_latitude 

In [30]:
# final_latitude_bins = np.arange(181) - 90

# final_longitude_bins = np.arange(361) -180




# H, _, _ = np.histogram2d(final_latitude, final_longitude, bins = (final_latitude_bins, final_longitude_bins), weights=volume)

# surface_area = abs(1750**2 * np.sin(np.deg2rad(90.5- final_latitude_bins[:-1])) *np.radians(1)* np.radians(1))
# surface_area = np.expand_dims(surface_area, axis=1)
# surface_area = np.repeat(surface_area, 360, axis=1) # make surface area same shape as H


# layer_thickness = H / (surface_area)
# print(layer_thickness.shape)

(180, 360)


Create output dataframe with filtered tracers and relevant ejecta parameters for plotting/analysis

In [22]:
def make_dataframe(tracer_idxs, ejection_times, v_r, longitude, latitude, theta_launch, height, distance, final_longitude, final_latitude, tracer_material, azimuth, initial_depth, pressure, volume):
    initial_df = pd.DataFrame({'Tracer Index' : tracer_idxs, 'Ejection Time' : ejection_times, 'Resultant Velocity' : v_r, 'Launch Longitude' : longitude[0], 'Launch Latitude' : latitude[0], 'Launch Angle' : theta_launch, 'Height' : height, 'Distance' : distance, 'Landing Longitude' : final_longitude, 'Landing Latitude' : final_latitude, 'Tracer Material' : tracer_material, 'Azimuth' : azimuth, 'Depth': initial_depth, 'Pressure': pressure, 'Volume': volume})
    return initial_df

In [23]:
df = make_dataframe(tracer_idxs, ejection_times, v_r, longitude, latitude, theta_launch, height, distance, final_longitude, final_latitude, tracer_material, azimuth, initial_depth, pressure, volume)
# print(df['Launch Longitude'][df['Launch Longitude'] < 150])
# print(df['Ejection Time'][df['Ejection Time'] > 0].min())
# print(df.Distance)
# print(df.Distance.describe())
# print(df[df.Distance < 0])
# # print(df['Ejection Time'].describe())
# print(df.loc[1360])
# print(df['Resultant Velocity'].describe())
# # print(df['Tracer Material'].value_counts())
# print(df['Resultant Velocity'].describe())

In [24]:
#remove all tracers with velocity greater than moon's escape velocity
df = df[df['Resultant Velocity'] > 2.38] 
df

,Tracer Index,Ejection Time,Resultant Velocity,Launch Longitude,Launch Latitude,Launch Angle,Height,Distance,Landing Longitude,Landing Latitude,Tracer Material,Azimuth,Depth,Pressure,Volume
10,10,160.010300,2.958980,179.104285,67.404634,57.960869,-5422.992798,3891.222992,-1.466237,59.995053,4.0,179.640865,-120.558186,2.023202e+11,125.000000
11,11,136.021805,2.871584,179.133634,68.329973,50.073515,-6138.454195,3856.313342,-2.463284,57.922032,4.0,178.948349,-125.548824,1.968362e+11,125.000000
15,15,234.039703,3.474619,176.606965,64.462790,32.473993,-4871.965522,2799.783514,-7.369841,27.150428,4.0,176.460486,-95.618410,2.249997e+11,125.000000
28,28,188.044403,3.191086,176.174668,66.251032,29.850496,-5464.254761,2756.651140,-3.825332,24.002954,4.0,180.000000,-110.616837,2.091911e+11,125.000000
49,56,160.010300,2.958980,179.104285,67.404634,57.960869,-5422.992798,3891.222992,-1.466237,59.995053,4.0,179.640865,-120.438345,2.028550e+11,125.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49315,499451,322.057800,2.658905,159.076186,64.282320,28.506389,-8814.024465,3016.611016,-23.755877,34.457535,2.0,177.637532,102.661948,5.026280e+10,358.881872
55162,517723,298.083313,2.549465,162.196485,64.472840,27.480267,-11219.094827,3055.421058,-110.359169,9.912251,2.0,92.010473,82.566677,1.220085e+11,222.728992
55912,519550,206.042694,1889.993837,160.133044,67.478923,55.012007,-2989.471228,3218.221029,-167.958911,-3.692402,2.0,33.162438,82.620468,6.930737e+10,221.230262
57365,523495,212.013199,2.439165,157.992306,68.083703,58.246881,-16120.785374,3666.057166,-79.888794,43.909365,2.0,135.189223,77.584981,1.037798e+11,223.682340


In [28]:
df_final = df[df['Tracer Material'] == 4.0]
df_final

,Tracer Index,Ejection Time,Resultant Velocity,Launch Longitude,Launch Latitude,Launch Angle,Height,Distance,Landing Longitude,Landing Latitude,Tracer Material,Azimuth,Depth,Pressure,Volume
10,10,160.010300,2.958980,179.104285,67.404634,57.960869,-5422.992798,3891.222992,-1.466237,59.995053,4.0,179.640865,-120.558186,2.023202e+11,125.0
11,11,136.021805,2.871584,179.133634,68.329973,50.073515,-6138.454195,3856.313342,-2.463284,57.922032,4.0,178.948349,-125.548824,1.968362e+11,125.0
15,15,234.039703,3.474619,176.606965,64.462790,32.473993,-4871.965522,2799.783514,-7.369841,27.150428,4.0,176.460486,-95.618410,2.249997e+11,125.0
28,28,188.044403,3.191086,176.174668,66.251032,29.850496,-5464.254761,2756.651140,-3.825332,24.002954,4.0,180.000000,-110.616837,2.091911e+11,125.0
49,56,160.010300,2.958980,179.104285,67.404634,57.960869,-5422.992798,3891.222992,-1.466237,59.995053,4.0,179.640865,-120.438345,2.028550e+11,125.0
50,57,136.021805,2.893446,179.136645,68.318769,50.814353,-5987.515319,3863.272497,-2.465338,58.161065,4.0,178.948972,-125.429302,1.972169e+11,125.0
51,58,136.021805,3.087127,179.076744,68.373774,72.373852,-4486.401690,3260.372332,-5.907852,38.306838,4.0,175.917046,-130.477006,1.898293e+11,125.0
52,59,136.021805,3.153717,179.112054,68.372072,73.357253,-4264.349132,3219.524200,-5.816663,36.971883,4.0,175.917060,-135.467976,1.804747e+11,125.0
62,72,136.021805,3.797692,177.701133,68.459868,61.385810,-3699.721329,3856.994400,-5.056473,57.803747,4.0,178.177742,-130.490296,1.887597e+11,125.0
63,73,136.021805,3.797692,177.799774,68.461192,61.384486,-3699.764914,3856.980575,-4.957741,57.801973,4.0,178.177724,-135.481231,1.803116e+11,125.0


In [77]:
# final_latitude_bins = np.arange(181) - 90

# final_longitude_bins = np.arange(361) -180




# H, _, _ = np.histogram2d(final_latitude, final_longitude, bins = (final_latitude_bins, final_longitude_bins), weights=volume)

# surface_area = abs(1750**2 * np.sin(np.deg2rad(90.5- final_latitude_bins[:-1])) *np.radians(1)* np.radians(1))
# surface_area = np.expand_dims(surface_area, axis=1)
# surface_area = np.repeat(surface_area, 360, axis=1) # make surface area same shape as H


# layer_thickness = H / (surface_area)


# print(layer_thickness.shape)

# print(stats.describe(layer_thickness[]))
# fig, ax = plt.subplots(figsize=(13, 4))
# final_plot = ax.imshow(layer_thickness, interpolation='nearest', origin='lower', extent=[-180, 180, -80, -60])
# # plt.xlabel('longs')
# # plt.ylabel('lats')

# cbar = plt.colorbar(final_plot, label='thickness (km) ', aspect=5)
# # cbar = plt.colorbar(layer_thickness, label='volume (km^3) ', aspect=5, cax = fig.add_axes([0.7, 0.2, 0.02, 0.6]))
# # cbar.minorticks_on()
# plt.savefig('thickness.png' , dpi = 600, facecolor = 'w', bbox_inches = 'tight')

In [31]:
# save the dataframe as a csv file 
df.to_csv(CSV_NAME + '.csv', index=False)    

In [32]:
#creating mirror data points to plot on lunar surface 

mirror = df.copy()
mirror.loc[:, 'Landing Longitude'] = -1 * mirror.loc[:, 'Landing Longitude']

In [33]:
frames = [df, mirror]
result = pd.concat(frames, ignore_index=True)
result

,Tracer Index,Ejection Time,Resultant Velocity,Launch Longitude,Launch Latitude,Launch Angle,Height,Distance,Landing Longitude,Landing Latitude,Tracer Material,Azimuth,Depth,Pressure,Volume
0,1,314.022797,0.679284,178.594896,64.980646,67.284500,158.805465,307.947670,-2.463628,-54.911890,3.0,176.522308,-83.204904,9.840282e+10,125.000000
1,4,540.002380,0.320635,177.521154,60.990997,35.253147,12.882374,284.626869,-3.203078,-51.680690,3.0,177.225807,-68.095904,1.072846e+11,125.000000
2,5,398.019287,1.188337,178.043452,63.326922,34.335207,287.041130,1627.917677,-2.546014,-10.029844,4.0,179.276017,-73.086451,1.042551e+11,125.000000
3,6,314.022797,0.679284,178.676098,64.981388,67.283759,158.803879,307.956582,-2.382468,-54.912340,4.0,176.522308,-78.077004,1.010291e+11,125.000000
4,7,314.022797,0.679284,178.594896,64.980646,67.284500,158.805465,307.947670,-2.463628,-54.911890,4.0,176.522308,-83.067569,9.840282e+10,125.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65867,524221,618.024414,0.046177,169.891817,60.865682,33.373780,0.234922,229.139978,10.108183,-53.363537,2.0,180.000000,105.000802,5.336313e+10,143.782851
65868,524250,484.032013,0.629364,174.946318,61.939530,38.507867,63.539719,497.107018,7.922620,-45.748348,2.0,172.840994,105.006904,4.810023e+10,71.860591
65869,524251,610.093994,0.216113,169.806314,60.883449,29.912509,4.292756,290.504755,14.558685,-51.680588,2.0,163.405773,104.999686,5.298057e+10,143.777123
65870,524280,482.013000,0.619038,174.794305,61.984149,30.629730,41.433450,526.915853,7.572901,-44.787736,2.0,174.327310,105.006060,4.796582e+10,71.894334


In [34]:
result.to_csv(CSV_NAME + '_mirror.csv', index=False)